In [1]:
import os, sys, glob, shutil
import argparse
import json
import yaml
import numpy as np
from pprint import pprint

import torch
import yaml
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from scipy.io import wavfile
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

In [2]:
%cd "D:\Schoolwork\TERM 3\WORK\visual_prosody"

D:\Schoolwork\TERM 3\WORK\visual_prosody


In [3]:
split_txt_val_path = r'.\preprocessed_data\Ego4D_final_v4\val.txt'
val_uids = []
with open(split_txt_val_path) as file:
    for line in file:
        # print(line.split('|')[0])
        val_uids.append(line.split('|')[0])

In [4]:
print(len(val_uids))

2641


In [5]:
from utils.model import get_model, get_vocoder, get_param_num, vocoder_infer
from utils.tools import to_device, log, synth_one_sample, expand, plot_mel
from model import FastSpeech2Loss
from dataset import Dataset
# from utils.auto_tqdm import tqdm
from evaluate import evaluate

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
parser = argparse.ArgumentParser()
parser.add_argument("--restore_step", type=int, default=0)
parser.add_argument(
    "-p",
    "--preprocess_config",
    type=str,
    required=True,
    help="path to preprocess.yaml",
)
parser.add_argument(
    "-m", "--model_config", type=str, required=True, help="path to model.yaml"
)
parser.add_argument(
    "-t", "--train_config", type=str, required=True, help="path to train.yaml"
)

argString = '-p ./config/Ego4D_final_v4/0712a_preprocess.yaml -m ./config/Ego4D_final_v4/0712a_model.yaml -t ./config/Ego4D_final_v4/0712a_train.yaml'
# args = parser.parse_args()
args = parser.parse_args(argString.split())

In [15]:
pprint(args)
# Read Config
preprocess_config = yaml.load(
    open(args.preprocess_config, "r"), Loader=yaml.FullLoader
)
model_config = yaml.load(open(args.model_config, "r"), Loader=yaml.FullLoader)
train_config = yaml.load(open(args.train_config, "r"), Loader=yaml.FullLoader)
configs = (preprocess_config, model_config, train_config)
print("Prepare training ...")

preprocess_config, model_config, train_config = configs

Namespace(restore_step=0, preprocess_config='./config/Ego4D_final_v4/0712a_preprocess.yaml', model_config='./config/Ego4D_final_v4/0712a_model.yaml', train_config='./config/Ego4D_final_v4/0712a_train.yaml')
Prepare training ...


In [16]:
ckpt_path = r'./output/0712a/ckpt/1000000.pth.tar'
ckpt = torch.load(ckpt_path)

In [17]:
# Prepare model
model, optimizer = get_model(args, configs, device, train=True)
model.load_state_dict(ckpt["model"], strict=False)
model.to(device)
model = nn.DataParallel(model)
num_param = get_param_num(model)
Loss = FastSpeech2Loss(preprocess_config, model_config).to(device)
print("Number of FastSpeech2 Parameters:", num_param)

=> Not using speaker embeddings.
False
None
Number of FastSpeech2 Parameters: 35159745


In [18]:
# Load vocoder
vocoder = get_vocoder(model_config, device)
step = args.restore_step + 1
model.eval()
print()
dataset = Dataset(
    "val.txt", 'val', preprocess_config, train_config, sort=False, drop_last=False
)

Removing weight norm...



In [19]:
batch_size = train_config["optimizer"]["batch_size"]
batch_size = 1

In [23]:
loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=dataset.collate_fn,
    )
# Get loss function
Loss = FastSpeech2Loss(preprocess_config, model_config).to(device)

In [24]:
os.path.join(train_config['path']['result_path'], 'plots')

'./output/0712a/result/Ego4D_final_v4\\plots'

In [28]:
output_plot_path = os.path.join(train_config['path']['result_path'], 'plot')
output_mel_syn_path = os.path.join(train_config['path']['result_path'], 'mel', 'syn')
output_mel_gt_path = os.path.join(train_config['path']['result_path'], 'mel', 'gt')
output_wav_syn_path = os.path.join(train_config['path']['result_path'], 'wav', 'synthesized')
output_wav_rec_path = os.path.join(train_config['path']['result_path'], 'wav', 'reconstructed')
os.makedirs(output_plot_path, exist_ok=True)
os.makedirs(output_mel_syn_path, exist_ok=True)
os.makedirs(output_mel_gt_path, exist_ok=True)
os.makedirs(output_wav_syn_path, exist_ok=True)
os.makedirs(output_wav_rec_path, exist_ok=True)

In [29]:
for batchs in tqdm(loader):
    for targets in batchs:
        targets = to_device(targets, device)
        with torch.no_grad():
            predictions = model(*(targets[2:]))
        basenames = targets[0]
        for i in range(len(predictions[0])):
            basename = basenames[i]
            src_len = predictions[8][i].item()
            mel_len = predictions[9][i].item()
            mel_prediction = predictions[1][i, :mel_len].detach().transpose(0, 1)
            mel_target = targets[6][i, :mel_len].detach().transpose(0, 1)

            torch.save(mel_prediction.cpu(), os.path.join(output_mel_syn_path, f"{basename}.pt"))
            torch.save(mel_target.cpu(), os.path.join(output_mel_gt_path, f"{basename}.pt"))
            
            
            duration = predictions[5][i, :src_len].detach().cpu().numpy()
            if preprocess_config["preprocessing"]["pitch"]["feature"] == "phoneme_level":
                pitch = predictions[2][i, :src_len].detach().cpu().numpy()
                pitch = expand(pitch, duration)
            else:
                pitch = predictions[2][i, :mel_len].detach().cpu().numpy()
            if preprocess_config["preprocessing"]["energy"]["feature"] == "phoneme_level":
                energy = predictions[3][i, :src_len].detach().cpu().numpy()
                energy = expand(energy, duration)
            else:
                energy = predictions[3][i, :mel_len].detach().cpu().numpy()

            with open(os.path.join(preprocess_config["path"]["preprocessed_path"], 
                                   "stats.json")) as f:
                stats = json.load(f)
                stats = stats["pitch"] + stats["energy"][:2]
                                       
            fig = plot_mel(
                [
                    (mel_prediction.cpu().numpy(), pitch, energy),
                    (mel_target.cpu().numpy(), pitch, energy),
                ],
                stats,
                ["Synthetized Spectrogram", "Ground-Truth Spectrogram"],
            )
            ### TODO: change to svg
            plt.savefig(os.path.join(output_plot_path, f"{basename}.png"))
            plt.close()

        # from .model import vocoder_infer

        mel_predictions = predictions[1].transpose(1, 2)
        mel_targets = targets[6].transpose(1, 2)
        
        lengths = predictions[9] * preprocess_config["preprocessing"]["stft"]["hop_length"]
        wav_predictions = vocoder_infer(
            mel_predictions, vocoder, model_config, preprocess_config, lengths=lengths
        )
        wav_targets = vocoder_infer(
        mel_targets, vocoder, model_config, preprocess_config, lengths=lengths
    )
    
        sampling_rate = preprocess_config["preprocessing"]["audio"]["sampling_rate"]
        for wav, basename in zip(wav_predictions, basenames):
            wavfile.write(os.path.join(output_wav_syn_path, f"{basename}.wav"), sampling_rate, wav)
        for wav, basename in zip(wav_targets, basenames):
            wavfile.write(os.path.join(output_wav_rec_path, f"{basename}.wav"), sampling_rate, wav)

        
    #     break
    # break

  0%|          | 0/2641 [00:00<?, ?it/s]

# MCD

In [30]:
import os, sys, glob, shutil
import os.path as op
import argparse
import json
import yaml
import numpy as np
import pandas as pd
from pprint import pprint
import os.path as op

import torch
import torch.nn.functional as F
import torchaudio
import argparse
import fnmatch
import logging
import multiprocessing as mp
from typing import Dict, List, Tuple

import librosa
import pysptk
import pyworld as pw
import soundfile as sf
from fastdtw import fastdtw
from scipy import spatial

In [31]:
split_txt_val_path = r'.\preprocessed_data\Ego4D_final_v4\val.txt'
val_uids = []
with open(split_txt_val_path) as file:
    for line in file:
        # print(line.split('|')[0])
        val_uids.append(line.split('|')[0])

In [32]:
len(val_uids)

2641

In [33]:
recon_folder = r'.\output\0712a\result\Ego4D_final_v4\wav\reconstructed'
syn_folder = r'.\output\0712a\result\Ego4D_final_v4\wav\synthesized'
gt_folder = r'.\raw_data\Ego4D_final_v4\val\Ego4D_final_v4'

In [34]:
%%time

def sptk_extract(
    x: np.ndarray,
    fs: int,
    n_fft: int = 512,
    n_shift: int = 256,
    mcep_dim: int = 25,
    mcep_alpha: float = 0.41,
    is_padding: bool = False,
) -> np.ndarray:
    """Extract SPTK-based mel-cepstrum.

    Args:
        x (ndarray): 1D waveform array.
        fs (int): Sampling rate
        n_fft (int): FFT length in point (default=512).
        n_shift (int): Shift length in point (default=256).
        mcep_dim (int): Dimension of mel-cepstrum (default=25).
        mcep_alpha (float): All pass filter coefficient (default=0.41).
        is_padding (bool): Whether to pad the end of signal (default=False).

    Returns:
        ndarray: Mel-cepstrum with the size (N, n_fft).

    """
    # perform padding
    if is_padding:
        n_pad = n_fft - (len(x) - n_fft) % n_shift
        x = np.pad(x, (0, n_pad), "reflect")

    # get number of frames
    n_frame = (len(x) - n_fft) // n_shift + 1

    # get window function
    win = pysptk.sptk.hamming(n_fft)

    # check mcep and alpha
    if mcep_dim is None or mcep_alpha is None:
        mcep_dim, mcep_alpha = _get_best_mcep_params(fs)

    # calculate spectrogram
    mcep = [
        pysptk.mcep(
            x[n_shift * i : n_shift * i + n_fft] * win,
            mcep_dim,
            mcep_alpha,
            eps=1e-6,
            etype=1,
        )
        for i in range(n_frame)
    ]

    return np.stack(mcep)


def _get_best_mcep_params(fs: int) -> Tuple[int, float]:
    if fs == 16000:
        return 23, 0.42
    elif fs == 22050:
        return 34, 0.45
    elif fs == 24000:
        return 34, 0.46
    elif fs == 44100:
        return 39, 0.53
    elif fs == 48000:
        return 39, 0.55
    else:
        raise ValueError(f"Not found the setting for {fs}.")





uids = []
MCDs_recon = []
for uid in tqdm(val_uids):
    uids.append(uid)
    gen_x, gen_fs = sf.read(op.join(recon_folder, f"{uid}.wav"), dtype="int16")
    gt_x, gt_fs = sf.read(op.join(gt_folder, f"{uid}.wav"), dtype="int16")
    # print(gen_fs, gt_fs)
    fs = gen_fs
    if gen_fs != gt_fs:
        gt_x = librosa.resample(gt_x.astype(float), orig_sr=gt_fs, target_sr=gen_fs)

    # extract ground truth and converted features
    gen_mcep = sptk_extract(
        x=gen_x,
        fs=fs,
        n_fft=512,
        n_shift=256,
        mcep_dim=None,
        mcep_alpha=None,
    )
    gt_mcep = sptk_extract(
        x=gt_x,
        fs=fs,
        n_fft=512,
        n_shift=256,
        mcep_dim=None,
        mcep_alpha=None,
    )
    
    # DTW
    _, path = fastdtw(gen_mcep, gt_mcep, dist=spatial.distance.euclidean)
    twf = np.array(path).T
    gen_mcep_dtw = gen_mcep[twf[0]]
    gt_mcep_dtw = gt_mcep[twf[1]]

    # MCD
    diff2sum = np.sum((gen_mcep_dtw - gt_mcep_dtw) ** 2, 1)
    mcd = np.mean(10.0 / np.log(10.0) * np.sqrt(2 * diff2sum), 0)
    result = mcd
    MCDs_recon.append(result)

  0%|          | 0/2641 [00:00<?, ?it/s]

CPU times: total: 11min 45s
Wall time: 14min 9s


In [35]:
%%time

def sptk_extract(
    x: np.ndarray,
    fs: int,
    n_fft: int = 512,
    n_shift: int = 256,
    mcep_dim: int = 25,
    mcep_alpha: float = 0.41,
    is_padding: bool = False,
) -> np.ndarray:
    """Extract SPTK-based mel-cepstrum.

    Args:
        x (ndarray): 1D waveform array.
        fs (int): Sampling rate
        n_fft (int): FFT length in point (default=512).
        n_shift (int): Shift length in point (default=256).
        mcep_dim (int): Dimension of mel-cepstrum (default=25).
        mcep_alpha (float): All pass filter coefficient (default=0.41).
        is_padding (bool): Whether to pad the end of signal (default=False).

    Returns:
        ndarray: Mel-cepstrum with the size (N, n_fft).

    """
    # perform padding
    if is_padding:
        n_pad = n_fft - (len(x) - n_fft) % n_shift
        x = np.pad(x, (0, n_pad), "reflect")

    # get number of frames
    n_frame = (len(x) - n_fft) // n_shift + 1

    # get window function
    win = pysptk.sptk.hamming(n_fft)

    # check mcep and alpha
    if mcep_dim is None or mcep_alpha is None:
        mcep_dim, mcep_alpha = _get_best_mcep_params(fs)

    # calculate spectrogram
    mcep = [
        pysptk.mcep(
            x[n_shift * i : n_shift * i + n_fft] * win,
            mcep_dim,
            mcep_alpha,
            eps=1e-6,
            etype=1,
        )
        for i in range(n_frame)
    ]

    return np.stack(mcep)


def _get_best_mcep_params(fs: int) -> Tuple[int, float]:
    if fs == 16000:
        return 23, 0.42
    elif fs == 22050:
        return 34, 0.45
    elif fs == 24000:
        return 34, 0.46
    elif fs == 44100:
        return 39, 0.53
    elif fs == 48000:
        return 39, 0.55
    else:
        raise ValueError(f"Not found the setting for {fs}.")





uids = []
MCDs_gt = []
for uid in tqdm(val_uids):
    uids.append(uid)
    gen_x, gen_fs = sf.read(op.join(syn_folder, f"{uid}.wav"), dtype="int16")
    gt_x, gt_fs = sf.read(op.join(gt_folder, f"{uid}.wav"), dtype="int16")
    # print(gen_fs, gt_fs)
    fs = gen_fs
    if gen_fs != gt_fs:
        gt_x = librosa.resample(gt_x.astype(float), orig_sr=gt_fs, target_sr=gen_fs)

    # extract ground truth and converted features
    gen_mcep = sptk_extract(
        x=gen_x,
        fs=fs,
        n_fft=512,
        n_shift=256,
        mcep_dim=None,
        mcep_alpha=None,
    )
    gt_mcep = sptk_extract(
        x=gt_x,
        fs=fs,
        n_fft=512,
        n_shift=256,
        mcep_dim=None,
        mcep_alpha=None,
    )
    
    # DTW
    _, path = fastdtw(gen_mcep, gt_mcep, dist=spatial.distance.euclidean)
    twf = np.array(path).T
    gen_mcep_dtw = gen_mcep[twf[0]]
    gt_mcep_dtw = gt_mcep[twf[1]]

    # MCD
    diff2sum = np.sum((gen_mcep_dtw - gt_mcep_dtw) ** 2, 1)
    mcd = np.mean(10.0 / np.log(10.0) * np.sqrt(2 * diff2sum), 0)
    result = mcd
    MCDs_gt.append(result)

  0%|          | 0/2641 [00:00<?, ?it/s]

CPU times: total: 11min 55s
Wall time: 13min 30s


In [36]:
v4_mcd_df = pd.DataFrame({
    'uid': uids,
    'MCD_recon': MCDs_recon,
    'MCD_gt': MCDs_gt,
})

In [37]:
v4_mcd_df.to_csv(r".\jupyter_walkthrough\metrics\MCD_0712a_1M_v4_new.csv")

In [39]:
print(f"MCDs_recon mean on 0712a_1M_v4: {np.nanmean(MCDs_recon)}")
print(f"MCDs_recon std on 0712a_1M_v4: {np.nanstd(MCDs_recon)}")

print(f"MCDs_gt mean on 0712a_1M_v4: {np.nanmean(MCDs_gt)}")
print(f"MCDs_gt std on 0712a_1M_v4: {np.nanstd(MCDs_gt)}")

MCDs_recon mean on 0712a_1M_v4: 4.710335337567078
MCDs_recon std on 0712a_1M_v4: 1.196364770343932
MCDs_gt mean on 0712a_1M_v4: 9.20648139192688
MCDs_gt std on 0712a_1M_v4: 1.5928134651664063


In [42]:
v4_mcd_df.nsmallest(5, 'MCD_recon')

,uid,MCD_recon,MCD_gt
2491,d6f99bd9-e9c7-41cd-8dfb-86b66871fc7e,2.997942,9.117190
1352,b6335079-b4e0-47a0-ae26-aeebe29f0441,3.033956,11.973094
2240,f08e023f-3d18-4243-8c17-26161d255030,3.056815,7.497515
465,273f6847-2c3c-44b4-8fb8-1e636d3cf6eb,3.059826,8.043425
715,4c848686-910e-41bd-b183-9a8867d236a9,3.121161,8.466061


In [43]:
v4_mcd_df.nlargest(5, 'MCD_recon')

,uid,MCD_recon,MCD_gt
1635,49a76e36-8573-45b9-9bf5-d27b8ea26dbb,15.321158,13.939626
2477,80eab8d0-a9ff-44ec-9778-3f35bf8020a1,12.842950,13.069163
432,e053504c-4a11-4ba4-b2b5-2be37ed6f5e2,12.733121,12.676626
681,3bdc5c8b-f30e-457a-83e4-6b179f0a0cd0,12.079830,11.678589
17,28f314aa-e11d-48b3-b897-2f0bd92943b7,11.818570,11.825336


# log f0

In [13]:
# https://github.com/espnet/espnet/blob/3e0dad524d62ccd45e067e9b36049f2214ea972a/egs2/TEMPLATE/asr1/pyscripts/utils/evaluate_f0.py

In [44]:
def world_extract(
    x: np.ndarray,
    fs: int,
    f0min: int = 40,
    f0max: int = 800,
    n_fft: int = 512,
    n_shift: int = 256,
    mcep_dim: int = 25,
    mcep_alpha: float = 0.41,
) -> np.ndarray:
    """Extract World-based acoustic features.

    Args:
        x (ndarray): 1D waveform array.
        fs (int): Minimum f0 value (default=40).
        f0 (int): Maximum f0 value (default=800).
        n_shift (int): Shift length in point (default=256).
        n_fft (int): FFT length in point (default=512).
        n_shift (int): Shift length in point (default=256).
        mcep_dim (int): Dimension of mel-cepstrum (default=25).
        mcep_alpha (float): All pass filter coefficient (default=0.41).

    Returns:
        ndarray: Mel-cepstrum with the size (N, n_fft).
        ndarray: F0 sequence (N,).

    """
    # extract features
    x = x.astype(np.float64)
    f0, time_axis = pw.harvest(
        x,
        fs,
        f0_floor=f0min,
        f0_ceil=f0max,
        frame_period=n_shift / fs * 1000,
    )
    sp = pw.cheaptrick(x, f0, time_axis, fs, fft_size=n_fft)
    if mcep_dim is None or mcep_alpha is None:
        mcep_dim, mcep_alpha = _get_best_mcep_params(fs)
    mcep = pysptk.sp2mc(sp, mcep_dim, mcep_alpha)

    return mcep, f0


def _get_basename(path: str) -> str:
    return os.path.splitext(os.path.split(path)[-1])[0]


def _get_best_mcep_params(fs: int) -> Tuple[int, float]:
    if fs == 16000:
        return 23, 0.42
    elif fs == 22050:
        return 34, 0.45
    elif fs == 24000:
        return 34, 0.46
    elif fs == 44100:
        return 39, 0.53
    elif fs == 48000:
        return 39, 0.55
    else:
        raise ValueError(f"Not found the setting for {fs}.")

In [45]:
uids = []
logf0_rmses_recon = []
logf0_corrs_recon = []

for uid in tqdm(val_uids):
    uids.append(uid)
    # load wav file as int16
    gen_x, gen_fs = sf.read(op.join(syn_folder, f"{uid}.wav"), dtype="int16")
    gt_x, gt_fs = sf.read(op.join(recon_folder, f"{uid}.wav"), dtype="int16")
    fs = gen_fs
    if gen_fs != gt_fs:
        gt_x = librosa.resample(gt_x.astype(float), orig_sr=gt_fs, target_sr=gen_fs)
    # extract ground truth and converted features
    gen_mcep, gen_f0 = world_extract(
        x=gen_x,
        fs=fs,
        f0min=40,
        f0max=800,
        n_fft=1024,
        n_shift=256,
        mcep_dim=None,
        mcep_alpha=None,
    )
    gt_mcep, gt_f0 = world_extract(
        x=gt_x,
        fs=fs,
        f0min=40,
        f0max=800,
        n_fft=1024,
        n_shift=256,
        mcep_dim=None,
        mcep_alpha=None,
    )
    
    # DTW
    _, path = fastdtw(gen_mcep, gt_mcep, dist=spatial.distance.euclidean)
    twf = np.array(path).T
    gen_f0_dtw = gen_f0[twf[0]]
    gt_f0_dtw = gt_f0[twf[1]]
    
    # Get voiced part
    nonzero_idxs = np.where((gen_f0_dtw != 0) & (gt_f0_dtw != 0))[0]
    gen_f0_dtw_voiced = np.log(gen_f0_dtw[nonzero_idxs])
    gt_f0_dtw_voiced = np.log(gt_f0_dtw[nonzero_idxs])

    # log F0 RMSE
    log_f0_rmse = np.sqrt(np.mean((gen_f0_dtw_voiced - gt_f0_dtw_voiced) ** 2))
    # print(f"{uid} {log_f0_rmse:.4f}")

    # log F0 corr
    log_f0_corr = np.corrcoef(gen_f0_dtw_voiced, gt_f0_dtw_voiced)[0][1]
    # print(f"{uid} {log_f0_corr:.4f}")

    logf0_rmses_recon.append(log_f0_rmse)
    logf0_corrs_recon.append(log_f0_corr)



  0%|          | 0/2641 [00:00<?, ?it/s]

D:\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2846: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
D:\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2705: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
D:\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2705: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
D:\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
D:\Programs\Python\Python311\Lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
D:\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
D:\Programs\Py

In [46]:
uids = []
logf0_rmses_gt = []
logf0_corrs_gt = []

for uid in tqdm(val_uids):
    uids.append(uid)
    # load wav file as int16
    gen_x, gen_fs = sf.read(op.join(syn_folder, f"{uid}.wav"), dtype="int16")
    gt_x, gt_fs = sf.read(op.join(gt_folder, f"{uid}.wav"), dtype="int16")
    fs = gen_fs
    if gen_fs != gt_fs:
        gt_x = librosa.resample(gt_x.astype(float), orig_sr=gt_fs, target_sr=gen_fs)
    # extract ground truth and converted features
    gen_mcep, gen_f0 = world_extract(
        x=gen_x,
        fs=fs,
        f0min=40,
        f0max=800,
        n_fft=1024,
        n_shift=256,
        mcep_dim=None,
        mcep_alpha=None,
    )
    gt_mcep, gt_f0 = world_extract(
        x=gt_x,
        fs=fs,
        f0min=40,
        f0max=800,
        n_fft=1024,
        n_shift=256,
        mcep_dim=None,
        mcep_alpha=None,
    )
    
    # DTW
    _, path = fastdtw(gen_mcep, gt_mcep, dist=spatial.distance.euclidean)
    twf = np.array(path).T
    gen_f0_dtw = gen_f0[twf[0]]
    gt_f0_dtw = gt_f0[twf[1]]
    
    # Get voiced part
    nonzero_idxs = np.where((gen_f0_dtw != 0) & (gt_f0_dtw != 0))[0]
    gen_f0_dtw_voiced = np.log(gen_f0_dtw[nonzero_idxs])
    gt_f0_dtw_voiced = np.log(gt_f0_dtw[nonzero_idxs])

    # log F0 RMSE
    log_f0_rmse = np.sqrt(np.mean((gen_f0_dtw_voiced - gt_f0_dtw_voiced) ** 2))
    # print(f"{uid} {log_f0_rmse:.4f}")

    # log F0 corr
    log_f0_corr = np.corrcoef(gen_f0_dtw_voiced, gt_f0_dtw_voiced)[0][1]
    # print(f"{uid} {log_f0_corr:.4f}")

    logf0_rmses_gt.append(log_f0_rmse)
    logf0_corrs_gt.append(log_f0_corr)

  0%|          | 0/2641 [00:00<?, ?it/s]

D:\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
D:\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [47]:
v4_logf0_df = pd.DataFrame({
    'uid': uids,
    'logf0_rmse_recon': logf0_rmses_recon,
    'logf0_corr_recon': logf0_corrs_recon,
    'logf0_rmse_gt': logf0_rmses_gt,
    'logf0_corr_gt': logf0_corrs_gt,
})
v4_logf0_df.to_csv(r".\jupyter_walkthrough\metrics\logF0_0712a_v4_1M_new.csv")

In [49]:
v4_logf0_df = v4_logf0_df.dropna()

In [51]:
print(f"logf0_rmse_recon mean on 0712a_v4_1M: {v4_logf0_df['logf0_rmse_recon'].values.mean()}")
print(f"logf0_rmse_recon std on 0712a_v4_1M: {v4_logf0_df['logf0_rmse_recon'].values.std()}")
print(f"logf0_corr_recon mean on 0712a_v4_1M: {v4_logf0_df['logf0_corr_recon'].values.mean()}")
print(f"logf0_corr_recon std on 0712a_v4_1M: {v4_logf0_df['logf0_corr_recon'].values.std()}")

print(f"logf0_rmse_gt mean on 0712a_v4_1M: {v4_logf0_df['logf0_rmse_gt'].values.mean()}")
print(f"logf0_rmse_gt std on 0712a_v4_1M: {v4_logf0_df['logf0_rmse_gt'].values.std()}")
print(f"logf0_corr_gt mean on 0712a_v4_1M: {v4_logf0_df['logf0_corr_gt'].values.mean()}")
print(f"logf0_corr_gt std on 0712a_v4_1M: {v4_logf0_df['logf0_corr_gt'].values.std()}")

logf0_rmse_recon mean on 0712a_v4_1M: 0.5508572610451278
logf0_rmse_recon std on 0712a_v4_1M: 0.3309150585063425
logf0_corr_recon mean on 0712a_v4_1M: 0.1335724405704627
logf0_corr_recon std on 0712a_v4_1M: 0.38795448299072294
logf0_rmse_gt mean on 0712a_v4_1M: 0.5688654798912349
logf0_rmse_gt std on 0712a_v4_1M: 0.33464315738411515
logf0_corr_gt mean on 0712a_v4_1M: 0.11785446972405585
logf0_corr_gt std on 0712a_v4_1M: 0.38337624020984956
